In [5]:
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies import MomentumTimeSeries, SmaCross
from src.utils import load_data


ticker = "UPST".upper()
period = "10y"
stock = yf.Ticker(ticker)
data = stock.history(period=period)
data = load_data("../data/XAU-USD.csv")

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [6]:
bt = Backtest(data, SmaCross, cash=10_000, commission=0.004)
stats = bt.run()
stats

Start                     2010-01-04 00:00:00
End                       2019-12-31 00:00:00
Duration                   3648 days 00:00:00
Exposure Time [%]                   76.629571
Equity Final [$]                   7892.18656
Equity Peak [$]                   10544.19304
Return [%]                         -21.078134
Buy & Hold Return [%]               35.447321
Return (Ann.) [%]                   -2.342997
Volatility (Ann.) [%]               10.841023
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -36.850677
Avg. Drawdown [%]                  -12.642925
Max. Drawdown Duration     2785 days 00:00:00
Avg. Drawdown Duration      931 days 00:00:00
# Trades                                   14
Win Rate [%]                        14.285714
Best Trade [%]                      17.924295
Worst Trade [%]                     -9.852027
Avg. Trade [%]                    

In [7]:
bt.plot()

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Row(id='1393', ...)

In [8]:
%%time
stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


CPU times: user 4.27 s, sys: 19.2 ms, total: 4.29 s
Wall time: 4.3 s


Start                     2010-01-04 00:00:00
End                       2019-12-31 00:00:00
Duration                   3648 days 00:00:00
Exposure Time [%]                   94.554849
Equity Final [$]                 13834.129123
Equity Peak [$]                  14757.938803
Return [%]                          38.341291
Buy & Hold Return [%]               35.447321
Return (Ann.) [%]                    3.304108
Volatility (Ann.) [%]               13.658151
Sharpe Ratio                         0.241915
Sortino Ratio                        0.359728
Calmar Ratio                         0.103379
Max. Drawdown [%]                  -31.961174
Avg. Drawdown [%]                   -4.401614
Max. Drawdown Duration     1514 days 00:00:00
Avg. Drawdown Duration      111 days 00:00:00
# Trades                                   45
Win Rate [%]                        37.777778
Best Trade [%]                      20.154726
Worst Trade [%]                     -8.380363
Avg. Trade [%]                    

In [9]:
%%time
bt = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0)
stats = bt.optimize(
    lookback=range(1,10,1),
    maximize='Equity Final [$]',
    # constraint=lambda param: param,
)
stats

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


CPU times: user 3.09 s, sys: 12.1 ms, total: 3.1 s
Wall time: 3.1 s


Start                     2010-01-04 00:00:00
End                       2019-12-31 00:00:00
Duration                   3648 days 00:00:00
Exposure Time [%]                   99.682035
Equity Final [$]                   24406.7656
Equity Peak [$]                    27644.8022
Return [%]                         144.067656
Buy & Hold Return [%]               35.447321
Return (Ann.) [%]                     9.34845
Volatility (Ann.) [%]               15.870308
Sharpe Ratio                         0.589053
Sortino Ratio                        0.936179
Calmar Ratio                         0.356514
Max. Drawdown [%]                  -26.221829
Avg. Drawdown [%]                   -2.572833
Max. Drawdown Duration      718 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                 2508
Win Rate [%]                        51.794258
Best Trade [%]                       5.426856
Worst Trade [%]                     -8.495958
Avg. Trade [%]                    

In [12]:
bt = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0, trade_on_close=True)
stats = bt.run()
stats

Start                     2010-01-04 00:00:00
End                       2019-12-31 00:00:00
Duration                   3648 days 00:00:00
Exposure Time [%]                   99.801272
Equity Final [$]                   17951.1929
Equity Peak [$]                    20076.1799
Return [%]                          79.511929
Buy & Hold Return [%]               35.447321
Return (Ann.) [%]                    6.035119
Volatility (Ann.) [%]               15.491005
Sharpe Ratio                         0.389589
Sortino Ratio                        0.619353
Calmar Ratio                         0.230684
Max. Drawdown [%]                  -26.161889
Avg. Drawdown [%]                   -4.796349
Max. Drawdown Duration      706 days 00:00:00
Avg. Drawdown Duration       78 days 00:00:00
# Trades                                 2511
Win Rate [%]                        50.099562
Best Trade [%]                       8.495958
Worst Trade [%]                     -5.074246
Avg. Trade [%]                    